In [1]:
import pdfplumber
import pandas as pd
import re
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.datavalidation import DataValidation


In [2]:
def extract_data_from_pdf(pdf_path):
    data1 = []  # Data for the first DataFrame
    data2 = []  # Data for the second DataFrame

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                valuation_date = ""
                for line in lines:
                    if "Valuation Date:" in line:
                        valuation_date_pattern = re.compile(r'Valuation Date: (\d{2}/\d{2}/\d{4})')
                        valuation_date_match = valuation_date_pattern.search(line)
                        if valuation_date_match:
                            valuation_date = valuation_date_match.group(1)
                            print(f"Found valuation date: {valuation_date}")
                    # Regex to match the data for the first DataFrame
                    match1 = re.match(r'(\d{2}/\d{2}/\d{4})-(\d{2}/\d{2}/\d{4})\s+\$(\d+(?:,\d{3})*(?:\.\d{2})?)\s+(\d+)', line)
                    if match1:
                        effective_date = match1.group(1)
                        expiration_date = match1.group(2)
                        total_incurred = match1.group(3).replace(',', '')
                        number = match1.group(4)
                        data1.append([effective_date, expiration_date, valuation_date, total_incurred, number])
                    
                    # Regex to match the data for the second DataFrame

                    match2 = re.match(r'(\d{2}/\d{2}/\d{4})\s+(\d+(?:,\d{3})*)\s+([A-Z])\s+(\d+(?:,\d{3})*)\s+(\d+(?:,\d{3})*)\s+([A-Z]{2})\s+(.*)', line)
                    if match2:
                        dol = match2.group(1)
                        incurred = match2.group(2).replace(',', '')
                        oc = match2.group(3)
                        paid = match2.group(4).replace(',', '')
                        reserve = match2.group(5).replace(',', '')
                        type_desc = match2.group(6)
                        description = match2.group(7)
                        data2.append([dol, incurred, oc, paid, reserve, type_desc, description])

    df1 = pd.DataFrame(data1, columns=['Effective Date', 'Expiration Date', 'Valuation Date', 'Total Incurred', 'Number'])
    df2 = pd.DataFrame(data2, columns=['DOL', 'Incurred', 'O/C', 'Paid', 'Reserve', 'Type', 'Description'])
    
    return df1, df2



In [3]:
template_path = 'C:\\Users\\asaxena\\Desktop\\Automation Project\\Loss Experience Template1.xlsx' 
#excel_path = "C:\\Users\\asaxena\\Desktop\\Automation Project\Updated\updated_excel_file.xls
#excel_path = 'C:\\Users\\asaxena\\Desktop\\Automation Project\\Loss Experience Template.xlsx
output_path = 'C:\\Users\\asaxena\\Desktop\\Automation Project\\Loss Experience Template.xlsx'
def save_to_excel(df1, df2, excel_path):
    try:
        wb = load_workbook(excel_path)
    except FileNotFoundError:
        wb = Workbook()


    # Load the workbook and select the active worksheet
    wb = load_workbook(template_path)   
    ws = wb.active

    # Write DataFrame 1 to Excel
    for i, row in enumerate(dataframe_to_rows(df1, index=False, header=False), start=9):
        ws.cell(row=i, column=3, value=row[0])  # Column C (Effective Date)
        ws.cell(row=i, column=4, value=row[1])  # Column D (Expiration Date)
        ws.cell(row=i, column=5, value=row[2])  # Column E (Valuation Date)
        ws.cell(row=i, column=6, value=row[3])  # Column G (Total Incurred)
        ws.cell(row=i, column=11, value=row[4])  # Column K (Number)

    # Write DataFrame 2 to Excel
    for i, row in enumerate(dataframe_to_rows(df2, index=False, header=False), start=9):
        ws.cell(row=i, column=14, value=row[0])  # Column N (DOL)
        ws.cell(row=i, column=15, value=row[1])  # Column O (Total Incurred)
        ws.cell(row=i, column=19, value=row[2])  # Column Q (Open/CLosed)
        ws.cell(row=i, column=17, value=row[3])  # Column S (Paid)

    # Add Data Validation for Open/Closed column in DataFrame 2
    dv = DataValidation(type="list", formula1='"Open,Closed"', showDropDown=True)
    ws.add_data_validation(dv)
    for cell in ws['S']:
        if cell.row >= 9:
            dv.add(cell)

    wb.save(excel_path)
    print(f"Data saved to {excel_path}")




In [4]:
def main():
    
    pdf_path='C:\\Users\\asaxena\\Desktop\\Automation Project\\GuyCarp submissions\\Fernlea Industries_ Inc__Submission_UMB_2024-06-04_012751_392.pdf'
    output_path ='C:\\Users\\asaxena\\Desktop\\Automation Project\\Loss Experience Template.xlsx'
    df1, df2 = extract_data_from_pdf(pdf_path)
    
    # Display the DataFrames
    print("DataFrame 1:")
    print(df1)
    print("\nDataFrame 2:")
    print(df2)
    
    save_to_excel(df1, df2, output_path)

if __name__ == "__main__":
    main()


Found valuation date: 05/07/2024
Found valuation date: 05/07/2024
DataFrame 1:
  Effective Date Expiration Date Valuation Date Total Incurred Number
0     06/01/2023      06/01/2024     05/07/2024              5      2
1     06/01/2022      06/01/2023     05/07/2024          44574      2
2     06/01/2021      06/01/2022     05/07/2024           2245      4
3     06/01/2020      06/01/2021     05/07/2024           3298      7
4     06/01/2019      06/01/2020     05/07/2024           4826      4
5     06/01/2018      06/01/2019     05/07/2024         106121      7

DataFrame 2:
          DOL Incurred O/C     Paid Reserve Type              Description
0  03/29/2021  2846123   C  2846123       0   UL  Insured vehicle backing
Data saved to C:\Users\asaxena\Desktop\Automation Project\Loss Experience Template.xlsx
